In [ ]:
import sys; sys.path.append('../')

In [ ]:
from common.load_ticker import load_ticker

df = load_ticker(
    base_dir = "../../../parquet_minute/", 
    time_col = "Date",
    symbol_col="Symbol",
    seed = 42, 
    symbol= "SPY",
    verbose=True
)

df

In [ ]:
from common.clean_data import clean_data
df = df.tail(100000)
df_1m = clean_data(
    df=df,
    timestamp_col = "Date",
    symbol_col = "Symbol",
    drop_duplicate_rows = True,
    drop_duplicate_cols = True,
    drop_constant_columns = True,
    drop_constant_rows = True,
    replace_placeholders = True,
    placeholders=("Null", "null", "NULL", "NaN", "nan", "NAN", "None", "none", "NONE"),
    fill_missing = True,
    convert_numeric = True,
    sort_by = "timestamp",
    verbose = True,
)

df_1m

In [ ]:
# import pandas as pd

# df_1m["timestamp"] = pd.to_datetime(df_1m["Date"])
# df_1m = df_1m.set_index("timestamp")

# # Function to resample into any timeframe
# def resample_ohlc(df, rule):
#     return df.resample(rule).agg({
#         "Open": "first",
#         "High": "max",
#         "Low": "min",
#         "Close": "last",
#         "Volume": "sum"
#     }).dropna()

# # Build higher TFs
# df_3m   = resample_ohlc(df_1m, "3T")
# df_5m   = resample_ohlc(df_1m, "5T")
# df_1h   = resample_ohlc(df_1m, "1H")
# df_4h   = resample_ohlc(df_1m, "4H")
# df_daily = resample_ohlc(df_1m, "1D")


In [ ]:
# # Assume you already have these built
# dfs = [
#     ("1m", df_1m),
#     ("3m", df_3m),
#     ("5m", df_5m),
#     ("1h", df_1h),
#     ("4h", df_4h),
#     ("1d", df_daily),
# ]


In [ ]:
# tf_strength = {
#     "1m": 1,
#     "3m": 0.2,
#     "5m": 0.3,
#     "1h": 0.4,
#     "4h": 0.5,
#     "1d": 0.6,
# }


In [ ]:
from charting.generate_features import generate_features
import pandas as pd

df_1m["timestamp"] = pd.to_datetime(df_1m["Date"])
df_1m = df_1m.set_index("timestamp")

# Function to resample into any timeframe
def resample_ohlc(df, rule):
    return df.resample(rule).agg({
        "Open": "first",
        "High": "max",
        "Low": "min",
        "Close": "last",
        "Volume": "sum"
    }).dropna()

# 1m already cleaned
# df_1m = generate_features(df_1m)

# Resample and generate features
df_3m   = generate_features(resample_ohlc(df_1m, "3min"))
df_5m   = generate_features(resample_ohlc(df_1m, "5min"))
df_15m   = generate_features(resample_ohlc(df_1m, "15min"))
df_1h   = generate_features(resample_ohlc(df_1m, "1H"))
# df_4h   = generate_features(resample_ohlc(df_1m, "4H"))
# df_daily = generate_features(resample_ohlc(df_1m, "1D"))

dfs = [
    # ("1m", df_1m),
    ("3m", df_3m),
    ("5m", df_5m),
    ("15m", df_15m),
    ("1h", df_1h),
    # ("4h", df_4h),
    # ("1D", df_daily),
]


In [ ]:
df_3m["ZoneLifecycle"][50]

In [ ]:
# import os

# SAVE_DIR = "./prepared_data"
# os.makedirs(SAVE_DIR, exist_ok=True)

# # Save each dataframe with its timeframe tag
# for name, df in dfs:
#     df.to_parquet(os.path.join(SAVE_DIR, f"{name}.parquet"))



In [ ]:
# from charting.generate_images import generate

# tf_strength = {
#     "1m": 0.5,
#     "5m": 1.0,
#     "1h": 2.0,
#     "4h": 3.0,
#     "1d": 5.0,
# }

# generate(
#     dfs=dfs,
#     support_dir="./dataset/support",
#     resistance_dir="./dataset/resistance",
#     recency_dir="./dataset/recency",
#     combined_dir="./dataset/combined",  
#     candles_dir="./dataset/candles",
#     window=3000,       
#     zone_extend=4000,   
#     right_padding=0.2, 
#     H=1200, W=1200,    
#     # tf_strength=tf_strength
# )


In [ ]:
import cv2
import os
import glob
import numpy as np
from datetime import datetime

def export_zone_video(
    output_file=None,
    fps=10,
    scale=0.7,
    dirs=None,
):
    """
    Export a side-by-side video of your zone charts.
    Order: Candles | Recency | Combined | Support | Resistance
    """

    # ✅ Auto-generate filename with timestamp if none provided
    if output_file is None:
        ts = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        output_file = f"./dataset/videos/zones_{ts}.mp4"

    # ✅ Ensure output directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    if dirs is None:
        dirs = {
            "Candles": "./dataset/candles",
            "Recency": "./dataset/recency",
            "Combined": "./dataset/combined",
            "Support": "./dataset/support",
            "Resistance": "./dataset/resistance",
        }
    order = ["Candles", "Recency", "Combined", "Support", "Resistance"]

    # Gather sorted file lists
    files = {name: sorted(glob.glob(os.path.join(d, "*.png"))) for name, d in dirs.items()}
    num_frames = min(len(lst) for lst in files.values() if lst)

    if num_frames == 0:
        raise ValueError("No frames found in one or more directories!")

    # Read first frame to fix video size
    sample = cv2.imread(files[order[0]][0])
    if sample is None:
        raise ValueError("Sample image could not be loaded.")

    # Normalize heights by using the minimum across dirs
    heights = []
    for name in order:
        img = cv2.imread(files[name][0])
        if img is not None:
            if scale != 1.0:
                h, w = img.shape[:2]
                img = cv2.resize(img, (int(w * scale), int(h * scale)))
            heights.append(img.shape[0])
    min_h = min(heights)

    # Width of stitched frame
    total_w = 0
    for name in order:
        img = cv2.imread(files[name][0])
        if img is None:
            continue
        if scale != 1.0:
            h, w = img.shape[:2]
            img = cv2.resize(img, (int(w * scale), int(h * scale)))
        img = cv2.resize(img, (int(img.shape[1] * min_h / img.shape[0]), min_h))
        total_w += img.shape[1]

    # Open writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_file, fourcc, fps, (total_w, min_h))

    # Frame loop
    for i in range(num_frames):
        frames = []
        for name in order:
            img = cv2.imread(files[name][i])
            if img is None:
                continue
            if scale != 1.0:
                h, w = img.shape[:2]
                img = cv2.resize(img, (int(w * scale), int(h * scale)))
            img = cv2.resize(img, (int(img.shape[1] * min_h / img.shape[0]), min_h))
            frames.append(img)

        stitched = np.hstack(frames)
        out.write(stitched)

    out.release()
    print(f"✅ Video saved: {output_file}")


# Example usage in notebook:
export_zone_video(fps=10, scale=0.6)


In [ ]:
# LOAD_DIR = "./prepared_data"

# dfs = []
# for name in ["1m", "3m", "5m", "15m", "1h", "4h", "1D"]:
#     df = pd.read_parquet(os.path.join(LOAD_DIR, f"{name}.parquet"))
#     df.index = pd.to_datetime(df.index)  # ensure index is datetime
#     dfs.append((name, df))


In [ ]:
# from charting.generate_c_images import generate

# generate(
#     dfs=dfs,
#     support_dir="./dataset/_c_support",
#     resistance_dir="./dataset/_c_resistance",
#     recency_dir="./dataset/_c_recency",
#     combined_dir="./dataset/_c_combined",  
#     candles_dir="./dataset/_c_candles",
#     window=2880,       
#     zone_extend=2880,   
#     right_padding=0.2, 
#     H=1200, W=1200,    
# )


In [ ]:
from charting.generate_d_images import generate

generate(
    dfs=dfs,
    # support_dir="./dataset/_d_support",
    # resistance_dir="./dataset/_d_resistance",
    # recency_dir="./dataset/_d_recency",
    combined_dir="./dataset/_d_combined",  
    candles_dir="./dataset/_d_candles",
    # window=1000,       
    # zone_extend=15000,   
    # right_padding=0.2, 
    H=640, W=640,    
)


In [ ]:
# import matplotlib.pyplot as plt

# # --- Colors in RGB (for matplotlib) ---
# TF_COLORS = {
#     "1d":  (255,   0,   0),   # Red
#     "4h":  (128,   0, 128),   # Purple
#     "1h":  (255, 255,   0),   # Yellow
#     "15m": (0, 255, 255),     # Teal
#     "5m":  (0, 200,   0),     # Green
#     "3m":  (255, 20, 147),    # Hot Pink
#     "1m":  (255, 255, 255),   # White
# }

# def plot_active_zones(dfs, idx=3000, window=300):
#     """
#     Quick test: Plot OHLC candles + active zones across timeframes at a given idx.
#     """
#     base_name, base_df = dfs[0]  # use the smallest TF (e.g. 1m) as base
#     win = base_df.iloc[idx-window:idx].copy()

#     fig, ax = plt.subplots(figsize=(12,6))
#     ax.set_facecolor("black")

#     # --- plot candlesticks (white/gray) ---
#     for i, row in enumerate(win.itertuples()):
#         color = "white" if row.Close >= row.Open else "gray"
#         ax.plot([i, i], [row.Low, row.High], color=color, linewidth=1)
#         ax.add_patch(
#             plt.Rectangle((i-0.3, min(row.Open, row.Close)),
#                           0.6,
#                           abs(row.Close-row.Open),
#                           facecolor=color)
#         )

#     # --- plot zones ---
#     for tf_name, df in dfs:
#         if idx >= len(df): 
#             continue
#         lifecycle = {}
#         for j in range(idx-window, idx):
#             if j < 0 or j >= len(df): 
#                 continue
#             for z in df.iloc[j].get("ZoneLifecycle", []):
#                 lifecycle[z["id"]] = z

#         for z in lifecycle.values():
#             if z.get("status") != "Active":
#                 continue
#             low, high = z["low"], z["high"]
#             if low is None or high is None: 
#                 continue

#             color = TF_COLORS.get(tf_name.lower(), (200,200,200))
#             hex_color = '#%02x%02x%02x' % color  # RGB → hex for matplotlib
#             ax.axhspan(low, high, color=hex_color, alpha=0.3,
#                        label=f"{tf_name} {z['type']}")

#     ax.set_title(f"Active Zones at frame {idx}")
#     ax.legend(loc="upper right")
#     plt.show()


In [ ]:
# plot_active_zones(dfs, idx=3000)
